In [1]:
using Catlab, Catlab.Theories
using Catlab.CategoricalAlgebra
using Catlab.Graphics
using Catlab.Graphics: Graphviz
import Catlab.CategoricalAlgebra: migrate!
using Catlab.WiringDiagrams
using Catlab.Programs
using Catlab.Programs.RelationalPrograms

In [2]:
draw(d::WiringDiagram) = to_graphviz(d,
    orientation=LeftToRight,
    labels=true, label_attr=:xlabel,
    node_attrs=Graphviz.Attributes(
      :fontname => "Courier",
    ),
    edge_attrs=Graphviz.Attributes(
      :fontname => "Courier",
    )
)

draw (generic function with 1 method)

In [3]:
# Form Workflow presentation of FreeBiproductCategory
@present Workflow(FreeBiproductCategory) begin
    (File,LRN,LPN,TypedLPN,StrataSpec,LPNss,ObsFunc,ParamVec,StateVec,TSpan,NumTS,SampleData,SampleTimes,ODEProb,ODESol,Labels,Loss)::Ob 
    LoadLRN::Hom(File,LRN)
    
    Homomorph::Hom(LPN⊗LPN,TypedLPN)
    StrataSpecify::Hom(TypedLPN⊗StrataSpec,LPNss)
    Stratify::Hom(LPNss⊗LPNss⊗LPN,LPN⊗ObsFunc) 

    GenerateData::Hom(LPN⊗ObsFunc⊗ParamVec⊗StateVec⊗TSpan⊗NumTS,SampleData⊗SampleTimes⊗ODEProb⊗ODESol⊗SampleData⊗Labels)
    Calibrate::Hom(LPN⊗ObsFunc⊗StateVec⊗ParamVec⊗SampleData⊗SampleTimes,ParamVec⊗ODESol⊗Loss)

    # FormControl
end

Presentation{ThBiproductCategory, Symbol}(Catlab.Theories.FreeBiproductCategory, (Ob = Catlab.Theories.FreeBiproductCategory.Ob{:generator}[File, LRN, LPN, TypedLPN, StrataSpec, LPNss, ObsFunc, ParamVec, StateVec, TSpan, NumTS, SampleData, SampleTimes, ODEProb, ODESol, Labels, Loss], Hom = Catlab.Theories.FreeBiproductCategory.Hom{:generator}[LoadLRN, Homomorph, StrataSpecify, Stratify, GenerateData, Calibrate]), Dict(:ODESol => (:Ob => 15), :Homomorph => (:Hom => 2), :ODEProb => (:Ob => 14), :StrataSpecify => (:Hom => 3), :SampleData => (:Ob => 12), :LRN => (:Ob => 2), :LPN => (:Ob => 3), :Labels => (:Ob => 16), :LoadLRN => (:Hom => 1), :LPNss => (:Ob => 6)…), Pair[])

In [4]:
# Form wiring diagram of load_stratify_calibrate_control Workflow
load_stratify_calibrate_control = @program Workflow (f_disease::File,f_strat::File,f_type::File,
                                                        spec_disease::StrataSpec,spec_strat::StrataSpec,
                                                        true_p::ParamVec,true_u0::StateVec,tspan::TSpan,num_ts::NumTS,
                                                        p_init::ParamVec) begin # 
    mdl_disease = LoadLRN(f_disease)
    mdl_strat = LoadLRN(f_strat)
    mdl_type = LoadLRN(f_type)

    # Form stratified model
    mdl_disease_typed = Homomorph(mdl_disease,mdl_type)
    mdl_strat_typed = Homomorph(mdl_strat,mdl_type)
    disease_ss = StrataSpecify(mdl_disease_typed,spec_disease)
    strat_ss = StrataSpecify(mdl_strat_typed,spec_strat)
    mdl_stratified, obs_func = Stratify(disease_ss,strat_ss,mdl_type) 

    # Simulate data
    sample_data, sample_times, prob_true, sol_true, noiseless_data, data_labels = GenerateData(mdl_stratified, obs_func, true_p, true_u0, tspan, num_ts)

    # Calibrate
    p_est, sol_est, loss = Calibrate(mdl_stratified, obs_func, true_u0, p_init, sample_data, sample_times)

    # Form controler

    return  p_est, sol_est, loss, mdl_stratified, obs_func, sample_data, sample_times, prob_true, sol_true, noiseless_data, data_labels
end

WiringDiagram{ThBiproductCategory}([:File,:File,:File,:StrataSpec,:StrataSpec,:ParamVec,:StateVec,:TSpan,:NumTS,:ParamVec], [:ParamVec,:ODESol,:Loss,:LPN,:ObsFunc,:SampleData,:SampleTimes,:ODEProb,:ODESol,:SampleData,:Labels], 
[ -2 => {inputs},
  -1 => {outputs},
  1 => Box(:LoadLRN, [:File], [:LRN]),
  2 => Box(:LoadLRN, [:File], [:LRN]),
  3 => Box(:LoadLRN, [:File], [:LRN]),
  4 => Box(:Homomorph, [:LPN,:LPN], [:TypedLPN]),
  5 => Box(:Homomorph, [:LPN,:LPN], [:TypedLPN]),
  6 => Box(:StrataSpecify, [:TypedLPN,:StrataSpec], [:LPNss]),
  7 => Box(:StrataSpecify, [:TypedLPN,:StrataSpec], [:LPNss]),
  8 => Box(:Stratify, [:LPNss,:LPNss,:LPN], [:LPN,:ObsFunc]),
  9 => Box(:GenerateData, [:LPN,:ObsFunc,:ParamVec,:StateVec,:TSpan,:NumTS], [:SampleData,:SampleTimes,:ODEProb,:ODESol,:SampleData,:Labels]),
  10 => Box(:Calibrate, [:LPN,:ObsFunc,:StateVec,:ParamVec,:SampleData,:SampleTimes], [:ParamVec,:ODESol,:Loss]) ],
[ Wire((-2,1) => (1,1)),
  Wire((-2,2) => (2,1)),
  Wire((-2,3) => (3,1

In [5]:
# Display wiring diagram of workflow
draw(load_stratify_calibrate_control)

Catlab.Graphics.Graphviz.Graph("G", true, "dot", Catlab.Graphics.Graphviz.Statement[Catlab.Graphics.Graphviz.Subgraph("", Catlab.Graphics.Graphviz.Statement[Catlab.Graphics.Graphviz.Node("n0in1", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in1")), Catlab.Graphics.Graphviz.Node("n0in2", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in2")), Catlab.Graphics.Graphviz.Node("n0in3", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in3")), Catlab.Graphics.Graphviz.Node("n0in4", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in4")), Catlab.Graphics.Graphviz.Node("n0in5", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in5")), Catlab.Graphics.Graphviz.Node("n0in6", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in6")), Cat

In [6]:
# Write diagram to file as JSON
write_json_acset(load_stratify_calibrate_control.diagram, "strat-calib-cntrl_dwd.json")

4778